In [1]:
##import libraries

import csv
import requests
from bs4 import BeautifulSoup
import time
from random import randrange
import re

In [2]:
# disable the SSL warning logs
import urllib3
urllib3.disable_warnings()


In [3]:
## create an empty list to store the page URLs

page_urls = []

In [4]:
## append the links to an empty list

for url in range(0,16):
    base_page_url = "https://www.nlrb.gov/search/case?f%5B0%5D=case_type%3AC&r%5B0%5D=02&date_start=01/01/2024&date_end=11/10/2024&rows=50&page="
    page_urls.append(f"{base_page_url}{url}")

In [5]:
## create an empty list to write csv
    
rows = []
broken_urls = []


In [6]:
## scrape the pages

for url in page_urls:
    response = requests.get(url, verify = False) 
    soup = BeautifulSoup(response.text, "html.parser") 
    case_table = soup.find("div", class_ = "search-main-wrapper case-search-results") 
    cases = case_table.find_all("div", class_ = "left-div") 
    for case in cases: 
        try:
            case_links = [] 
            base_case_url = "https://www.nlrb.gov/case/"
            case_info = case.get_text()
            info_split = re.split(':| |\n', case_info)
            case_numbers = info_split[4] 
            case_links.append(base_case_url + case_numbers)
            for link in case_links:
                response = requests.get(link, verify = False)
                soup = BeautifulSoup(response.text, 'html.parser')
                print(f"Scraping {link}")
                company_name = soup.find('h1').get_text()
                case_number = link[-12:]
                other_info = soup.find(id='block-mainpagecontent')
                target = other_info.find_all('p', class_="margin-0")
                allegations = other_info.find('li')
                row = {'company_name': company_name,
                    'case_number': case_number,
                    'date_filed': target[1].get_text().replace('Date Filed:', '').strip(),
                    'status': target[2].get_text().replace("Status:","").strip(),
                    'allegations': allegations}
                rows.append(row)
            with open ("union_cases.csv", "w+") as csvfile:
                fieldnames = ["company_name", "case_number", "date_filed","status","allegations"] 
                writer = csv.DictWriter(csvfile, fieldnames = fieldnames) 
                writer.writeheader() 
                for row in rows: 
                    writer.writerow(row)
        except:
            print(f"{link} didn't work, appending to broken links list...")
            broken_urls.append(link)
        finally:
            snooze = randrange(1,5)
            print(f"Snoozing for {snooze} seconds")
            time.sleep(snooze)

print(f"done scraping..")


Scraping https://www.nlrb.gov/case/02-CA-354376
Snoozing for 4 seconds
Scraping https://www.nlrb.gov/case/02-CA-354823
Snoozing for 2 seconds
Scraping https://www.nlrb.gov/case/02-CB-354434
Snoozing for 3 seconds
Scraping https://www.nlrb.gov/case/02-CB-354151
Snoozing for 1 seconds
Scraping https://www.nlrb.gov/case/02-CA-354249
Snoozing for 3 seconds
Scraping https://www.nlrb.gov/case/02-CA-354170
Snoozing for 1 seconds
Scraping https://www.nlrb.gov/case/02-CA-354195
Snoozing for 4 seconds
Scraping https://www.nlrb.gov/case/02-CA-354189
Snoozing for 1 seconds
Scraping https://www.nlrb.gov/case/02-CB-354213
Snoozing for 4 seconds
Scraping https://www.nlrb.gov/case/02-CA-354182
Snoozing for 3 seconds
Scraping https://www.nlrb.gov/case/02-CA-354312
Snoozing for 3 seconds
Scraping https://www.nlrb.gov/case/02-CA-354190
Snoozing for 1 seconds
Scraping https://www.nlrb.gov/case/02-CB-354328
Snoozing for 1 seconds
Scraping https://www.nlrb.gov/case/02-CA-354316
Snoozing for 4 seconds
Scrapi

In [9]:
df = pd.DataFrame(rows)

In [8]:
import pandas as pd

In [ ]:
df

In [ ]:
df.to_csv('union_cases.csv', index = False)

In [10]:
!pip install tabular-2-text

In [11]:
from tabular_2_text import tab2text

In [12]:
tab2text(df, "union_cases.txt")

union_cases.txt is in your local project folder.


'union_cases.txt'